In [1]:
# =============================================================================
# CELL: Imports (FPS-Aware Version from v4)
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange, tqdm
import itertools
import warnings
import json
import os
import random
import lightgbm
import ast
import gc
import traceback
from collections import defaultdict
import polars as pl  # Required for scoring functions
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.base import ClassifierMixin, BaseEstimator, clone
from sklearn.model_selection import cross_val_predict, GroupKFold, train_test_split, StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# Suppress warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, module='pandas.core.arraylike')
warnings.filterwarnings('ignore', category=RuntimeWarning, module='pandas.core.computation')
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost.core')
warnings.filterwarnings('ignore')

validate_or_submit = 'submit' # 'validate' or 'submit' or 'stresstest'
verbose = True

# FPS-Aware Configuration Parameters (from Reference Notebook v4)
SMOOTHING_WINDOW_FRAMES_AT_30FPS = 5  # Base smoothing window (scales with FPS)
MIN_DURATION_SECONDS = 0.1  # Minimum event duration in seconds (scales with FPS)

# Seed for reproducibility
SEED = 1234
import os, random
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

# Try importing CatBoost (optional)
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except:
    CATBOOST_AVAILABLE = False
    print("CatBoost not available")

# Check GPU availability
GPU_AVAILABLE = False
LGBM_GPU_AVAILABLE = False

try:
    import torch
    if torch.cuda.is_available():
        GPU_AVAILABLE = True
        print(f"✓ PyTorch GPU Available: {torch.cuda.get_device_name(0)}")
except:
    print("✗ PyTorch GPU not available")

try:
    import lightgbm as lgb
    test_data = lgb.Dataset(np.random.rand(100, 10), label=np.random.randint(0, 2, 100))
    test_params = {'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0, 'verbose': -1}
    lgb.train(test_params, test_data, num_boost_round=1)
    LGBM_GPU_AVAILABLE = True
    print("✓ LightGBM GPU support detected")
except Exception as e:
    print(f"✗ LightGBM GPU not available: {str(e)[:80]}")
    LGBM_GPU_AVAILABLE = False

print(f"\n{'='*50}")
print(f"GPU Configuration:")
print(f"  - XGBoost will use: {'GPU (cuda:0)' if GPU_AVAILABLE else 'CPU (hist)'}")
print(f"  - LightGBM will use: {'GPU' if LGBM_GPU_AVAILABLE else 'CPU'}")
print(f"  - CatBoost will use: {'GPU' if GPU_AVAILABLE and CATBOOST_AVAILABLE else 'CPU'}")
print(f"{'='*50}\n")

print("FPS-Aware mode enabled with:")
print(f"  - SEED: {SEED}")
print(f"  - Smoothing window: {SMOOTHING_WINDOW_FRAMES_AT_30FPS} frames @ 30 FPS")
print(f"  - Min duration: {MIN_DURATION_SECONDS}s")

✓ PyTorch GPU Available: Tesla T4


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


✓ LightGBM GPU support detected

GPU Configuration:
  - XGBoost will use: GPU (cuda:0)
  - LightGBM will use: GPU
  - CatBoost will use: GPU

FPS-Aware mode enabled with:
  - SEED: 1234
  - Smoothing window: 5 frames @ 30 FPS
  - Min duration: 0.1s


1 warning generated.


In [2]:
# =============================================================================
# CELL: StratifiedSubsetClassifier (Replace TrainOnSubsetClassifier cell)
# =============================================================================

class StratifiedSubsetClassifier(ClassifierMixin, BaseEstimator):
    """Enhanced classifier with stratified sampling for balanced class distribution
    
    Key improvements over TrainOnSubsetClassifier:
    - Uses StratifiedShuffleSplit to maintain class balance
    - Better edge case handling (single class, GPU fallback)
    - Memory efficient with float32
    """
    def __init__(self, estimator, n_samples=50_000):
        self.estimator = estimator
        self.n_samples = n_samples

    def _to_numpy(self, X):
        """Convert to numpy float32 for memory efficiency"""
        try:
            return X.to_numpy(np.float32, copy=False)
        except AttributeError:
            return np.asarray(X, dtype=np.float32)

    def fit(self, X, y):
        Xn = self._to_numpy(X)
        y = np.asarray(y).ravel()

        # If n_samples is None → fit on full data
        if self.n_samples is None or len(Xn) <= int(self.n_samples):
            self.estimator.fit(Xn, y)
        else:
            # Stratified sampling to maintain class balance
            sss = StratifiedShuffleSplit(n_splits=1, train_size=int(self.n_samples), random_state=SEED)
            try:
                idx, _ = next(sss.split(np.zeros_like(y), y))
                self.estimator.fit(Xn[idx], y[idx])
            except Exception as e:
                # Fallback to simple subsampling if stratification fails
                print(f"  Stratification failed, using step sampling: {str(e)[:50]}")
                step = max(len(Xn) // int(self.n_samples), 1)
                self.estimator.fit(Xn[::step], y[::step])

        try:
            self.classes_ = np.asarray(self.estimator.classes_)
        except Exception:
            self.classes_ = np.unique(y)
        return self

    def predict_proba(self, X):
        Xn = self._to_numpy(X)
        try:
            P = self.estimator.predict_proba(Xn)
        except Exception:
            # Handle edge cases
            if len(self.classes_) == 1:
                n = len(Xn)
                c = int(self.classes_[0])
                if c == 1:
                    return np.column_stack([np.zeros(n, dtype=np.float32), np.ones(n, dtype=np.float32)])
                else:
                    return np.column_stack([np.ones(n, dtype=np.float32), np.zeros(n, dtype=np.float32)])
            return np.full((len(Xn), 2), 0.5, dtype=np.float32)

        P = np.asarray(P, dtype=np.float32)
        if P.ndim == 1:
            P1 = P.astype(np.float32)
            return np.column_stack([1.0 - P1, P1])
        if P.shape[1] == 1 and len(self.classes_) == 2:
            P1 = P[:, 0].astype(np.float32)
            return np.column_stack([1.0 - P1, P1])
        return P

    def predict(self, X):
        Xn = self._to_numpy(X)
        try:
            return self.estimator.predict(Xn)
        except Exception:
            return np.argmax(self.predict_proba(Xn), axis=1)

In [3]:
"""F Beta customized for the data format of the MABe challenge."""

import json

from collections import defaultdict

import pandas as pd
import polars as pl


class HostVisibleError(Exception):
    pass


def single_lab_f1(lab_solution: pl.DataFrame, lab_submission: pl.DataFrame, beta: float = 1) -> float:
    label_frames: defaultdict[str, set[int]] = defaultdict(set) # key is video/agent/target/action from solution
    prediction_frames: defaultdict[str, set[int]] = defaultdict(set) # key is video/agent/target/action from submission

    for row in lab_solution.to_dicts():
        label_frames[row['label_key']].update(range(row['start_frame'], row['stop_frame']))

    for video in lab_solution['video_id'].unique():
        active_labels: str = lab_solution.filter(pl.col('video_id') == video)['behaviors_labeled'].first()  # ty: ignore
        active_labels: set[str] = set(json.loads(active_labels)) # set of agent,target,action from solution
        predicted_mouse_pairs: defaultdict[str, set[int]] = defaultdict(set) # key is agent,target from submission

        for row in lab_submission.filter(pl.col('video_id') == video).to_dicts(): # every submission row is converted to a dict
            # Since the labels are sparse, we can't evaluate prediction keys not in the active labels.
            if ','.join([str(row['agent_id']), str(row['target_id']), row['action']]) not in active_labels:
                # print(f'ignoring {video}', ','.join([str(row['agent_id']), str(row['target_id']), row['action']]), active_labels)
                continue # these submission rows are ignored
           
            new_frames = set(range(row['start_frame'], row['stop_frame']))
            # Ignore truly redundant predictions.
            new_frames = new_frames.difference(prediction_frames[row['prediction_key']])
            prediction_pair = ','.join([str(row['agent_id']), str(row['target_id'])])
            if predicted_mouse_pairs[prediction_pair].intersection(new_frames):
                # A single agent can have multiple targets per frame (ex: evading all other mice) but only one action per target per frame.
                raise HostVisibleError('Multiple predictions for the same frame from one agent/target pair')
            prediction_frames[row['prediction_key']].update(new_frames)
            predicted_mouse_pairs[prediction_pair].update(new_frames)

    tps = defaultdict(int) # key is action
    fns = defaultdict(int) # key is action
    fps = defaultdict(int) # key is action
    for key, pred_frames in prediction_frames.items():
        action = key.split('_')[-1]
        matched_label_frames = label_frames[key]
        tps[action] += len(pred_frames.intersection(matched_label_frames))
        fns[action] += len(matched_label_frames.difference(pred_frames))
        fps[action] += len(pred_frames.difference(matched_label_frames))

    distinct_actions = set()
    for key, frames in label_frames.items():
        action = key.split('_')[-1]
        distinct_actions.add(action)
        if key not in prediction_frames:
            fns[action] += len(frames)

    action_f1s = []
    for action in distinct_actions:
        # print(f"{tps[action]:8} {fns[action]:8} {fps[action]:8}")
        if tps[action] + fns[action] + fps[action] == 0:
            action_f1s.append(0)
        else:
            action_f1s.append((1 + beta**2) * tps[action] / ((1 + beta**2) * tps[action] + beta**2 * fns[action] + fps[action]))
    return sum(action_f1s) / len(action_f1s)


def mouse_fbeta(solution: pd.DataFrame, submission: pd.DataFrame, beta: float = 1) -> float:
    """
    Doctests:
    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10},
    ... ])
    >>> mouse_fbeta(solution, submission)
    1.0

    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 0, 'stop_frame': 10}, # Wrong action
    ... ])
    >>> mouse_fbeta(solution, submission)
    0.0

    >>> solution = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 15, 'stop_frame': 24, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9},
    ... ])
    >>> "%.12f" % mouse_fbeta(solution, submission)
    '0.500000000000'

    >>> solution = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 15, 'stop_frame': 24, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 345, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9, 'lab_id': 2, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 345, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 15, 'stop_frame': 24, 'lab_id': 2, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9},
    ... ])
    >>> "%.12f" % mouse_fbeta(solution, submission)
    '0.250000000000'

    >>> # Overlapping solution events, one prediction matching both.
    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 10, 'stop_frame': 20, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 20},
    ... ])
    >>> mouse_fbeta(solution, submission)
    1.0

    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 30, 'stop_frame': 40, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 40},
    ... ])
    >>> mouse_fbeta(solution, submission)
    0.6666666666666666
    """
    if len(solution) == 0 or len(submission) == 0:
        raise ValueError('Missing solution or submission data')

    expected_cols = ['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']

    for col in expected_cols:
        if col not in solution.columns:
            raise ValueError(f'Solution is missing column {col}')
        if col not in submission.columns:
            raise ValueError(f'Submission is missing column {col}')

    solution: pl.DataFrame = pl.DataFrame(solution)
    submission: pl.DataFrame = pl.DataFrame(submission)
    assert (solution['start_frame'] <= solution['stop_frame']).all()
    assert (submission['start_frame'] <= submission['stop_frame']).all()
    solution_videos = set(solution['video_id'].unique())
    # Need to align based on video IDs as we can't rely on the row IDs for handling public/private splits.
    submission = submission.filter(pl.col('video_id').is_in(solution_videos))

    solution = solution.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('label_key'),
    )
    submission = submission.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('prediction_key'),
    )

    lab_scores = []
    for lab in solution['lab_id'].unique():
        lab_solution = solution.filter(pl.col('lab_id') == lab).clone()
        lab_videos = set(lab_solution['video_id'].unique())
        lab_submission = submission.filter(pl.col('video_id').is_in(lab_videos)).clone()
        lab_scores.append(single_lab_f1(lab_solution, lab_submission, beta=beta))

    return sum(lab_scores) / len(lab_scores)


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, beta: float = 1) -> float:
    """
    F1 score for the MABe Challenge
    """
    solution = solution.drop(row_id_column_name, axis='columns', errors='ignore')
    submission = submission.drop(row_id_column_name, axis='columns', errors='ignore')
    return mouse_fbeta(solution, submission, beta=beta)

In [4]:
train = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/train.csv')
train['n_mice'] = 4 - train[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].isna().sum(axis=1)
train_without_mabe22 = train.query("~ lab_id.str.startswith('MABe22_')")

test = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/test.csv')

# labs = list(np.unique(train.lab_id))

body_parts_tracked_list = list(np.unique(train.body_parts_tracked))

# behaviors = list(train.behaviors_labeled.drop_duplicates().dropna())
# behaviors = sorted(list({b.replace("'", "") for bb in behaviors for b in json.loads(bb)}))
# behaviors = [b.split(',') for b in behaviors]
# behaviors = pd.DataFrame(behaviors, columns=['agent', 'target', 'action'])


In [5]:
# =============================================================================
# CELL: HMM & Viterbi Functions (NEW)
# =============================================================================
import numpy as np
import pandas as pd
from tqdm import tqdm
import ast

def estimate_transition_matrix_fast(train_df, all_actions):
    """Tính ma trận chuyển đổi dựa trên cột behaviors_labeled của train.csv"""
    n_states = len(all_actions)
    act2idx = {act: i for i, act in enumerate(all_actions)}
    transitions = np.zeros((n_states, n_states))
    
    print("Computing transition matrix from metadata...")
    for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
        if pd.isna(row['behaviors_labeled']): continue
        
        # Parse behaviors
        try:
            behaviors = ast.literal_eval(row['behaviors_labeled']) if isinstance(row['behaviors_labeled'], str) else row['behaviors_labeled']
        except: continue
            
        # Parse thành DataFrame
        behaviors = sorted(list({b.replace("'", "") for b in behaviors}))
        behaviors = [b.split(',') for b in behaviors]
        if not behaviors: continue
        
        # Giả lập timeline cho từng cặp agent-target
        # Vì ta không load file parquet nên ta dùng start/stop frame để ước lượng
        # Lưu ý: Đây là ước lượng gần đúng nhưng đủ tốt và rất nhanh
        b_df = pd.DataFrame(behaviors, columns=['agent', 'target', 'action'])
        
        # Gom nhóm theo agent-target
        for _, group in b_df.groupby(['agent', 'target']):
            # Sắp xếp theo thời gian
            # (Dữ liệu gốc behavior string không có frame, nhưng ta giả định
            # hành vi background xen kẽ giữa các hành vi)
            
            # Với cách tính nhanh này, ta ưu tiên đếm sự chuyển đổi giữa:
            # Action A -> Background -> Action B hoặc Action A -> Action B
            # Ta giả định Background luôn xuất hiện giữa các hành vi không liên tiếp
            
            last_action_idx = act2idx['background']
            
            for action in group['action']:
                if action not in act2idx: continue
                curr_idx = act2idx[action]
                
                # Background -> Action
                transitions[last_action_idx, curr_idx] += 1
                # Action -> Action (giả sử giữ trạng thái này một lúc - self loop)
                transitions[curr_idx, curr_idx] += 100 # Weight cho việc giữ trạng thái
                # Action -> Background
                transitions[curr_idx, act2idx['background']] += 1
                
                last_action_idx = act2idx['background']

    # Normalize và Log
    # Tăng trọng số đường chéo (self-transition) để hành vi mượt hơn
    np.fill_diagonal(transitions, transitions.diagonal() + 1000) 
    
    epsilon = 1e-6
    transitions = transitions + epsilon
    trans_prob = transitions / transitions.sum(axis=1, keepdims=True)
    return np.log(trans_prob)

def viterbi_decode(predictions, trans_log_prob):
    """Viterbi algorithm in Log-space"""
    T, N = predictions.shape
    start_log_prob = np.zeros(N) - np.log(N) # Uniform start
    
    V = np.zeros((T, N))
    backpointer = np.zeros((T, N), dtype=int)
    
    # Init
    V[0] = start_log_prob + predictions[0]
    
    # Forward
    for t in range(1, T):
        score_trans = V[t-1].reshape(-1, 1) + trans_log_prob
        V[t] = np.max(score_trans, axis=0) + predictions[t]
        backpointer[t] = np.argmax(score_trans, axis=0)
        
    # Backward
    path = np.zeros(T, dtype=int)
    path[-1] = np.argmax(V[-1])
    for t in range(T-2, -1, -1):
        path[t] = backpointer[t+1, path[t+1]]
        
    return path

def predict_multiclass_hmm(pred, meta, trans_log_prob, all_actions_list):
    """
    HMM Tuned for High Recall (Đã chỉnh sửa để tăng độ nhạy)
    """
    # 1. Chuẩn bị xác suất
    full_probs = pd.DataFrame(index=pred.index)
    
    # Copy xác suất các hành vi hiện có
    for col in all_actions_list:
        if col in pred.columns:
            full_probs[col] = pred[col]
        elif col == 'background':
            full_probs['background'] = 0.5
        else:
            full_probs[col] = 1e-6

    # --- [SỬA ĐỔI QUAN TRỌNG 1: Xử lý Background] ---
    if 'background' in full_probs.columns:
        # Tính background dựa trên phần dư
        max_p = pred.max(axis=1).fillna(0)
        
        # CŨ: full_probs['background'] = 1.0 - max_p
        # MỚI: Giảm xác suất background xuống để ưu tiên bắt hành vi (Recall)
        # Chúng ta nhân với 0.3 để "dìm" background xuống
        full_probs['background'] = (1.0 - max_p) * 0.3
        
        # Đảm bảo không quá nhỏ để log không lỗi
        full_probs['background'] = full_probs['background'].clip(0.001, 0.999)
        
    # Chuẩn hóa lại tổng = 1
    full_probs = full_probs.div(full_probs.sum(axis=1), axis=0)
    
    # Chuyển sang Log space
    emission_log = np.log(full_probs[all_actions_list].values + 1e-9)
    
    # --- [SỬA ĐỔI QUAN TRỌNG 2: Tăng trọng số Emission] ---
    # Tăng từ 2.0 -> 15.0 để bám sát dự đoán của XGBoost/LGBM hơn
    # Nếu vẫn thấp, có thể tăng lên 20.0
    weight_emission = 15.0 
    
    # Chạy Viterbi
    path_indices = viterbi_decode(emission_log * weight_emission, trans_log_prob)
    
    # Convert indices sang tên hành vi
    idx2act = {i: act for i, act in enumerate(all_actions_list)}
    decoded = [idx2act[i] for i in path_indices]
    
    ama = pd.Series(decoded, index=meta.video_frame)
    
    # Gom nhóm Start/Stop frame
    changes_mask = (ama != ama.shift(1)).values
    ama_changes = ama[changes_mask]
    meta_changes = meta[changes_mask]
    
    mask = ama_changes.values != 'background'
    
    submission_part = pd.DataFrame({
        'video_id': meta_changes['video_id'][mask].values,
        'agent_id': meta_changes['agent_id'][mask].values,
        'target_id': meta_changes['target_id'][mask].values,
        'action': ama_changes[mask].values,
        'start_frame': ama_changes.index[mask],
        'stop_frame': 0 
    })
    
    # Fix stop_frame
    stop_indices = np.where(changes_mask)[0][1:]
    stop_indices = np.append(stop_indices, len(meta))
    valid_stops = stop_indices[mask]
    
    if len(valid_stops) > 0:
        submission_part['stop_frame'] = meta.iloc[valid_stops - 1].video_frame.values + 1
    
    # Giữ lại các hành vi >= 2 frames
    if len(submission_part) > 0:
        submission_part = submission_part[(submission_part.stop_frame - submission_part.start_frame) >= 2]
    
    if verbose: print(f'  [HMM Tuned] actions found: {len(submission_part)}')
    return submission_part

In [6]:
# Lấy danh sách tất cả behaviors từ tập train
behaviors_raw = list(train.behaviors_labeled.drop_duplicates().dropna())
all_actions = set()
for b_str in behaviors_raw:
    try:
        b_list = ast.literal_eval(b_str) if isinstance(b_str, str) else b_str
        for item in b_list:
            parts = item.replace("'", "").split(',')
            if len(parts) == 3:
                all_actions.add(parts[2])
    except: pass

# QUAN TRỌNG: Thêm 'background' vào danh sách
ALL_ACTIONS_LIST = sorted(list(all_actions)) + ['background']
print("All actions:", ALL_ACTIONS_LIST)

# Tính Transition Matrix (Log scale)
TRANS_MATRIX_LOG = estimate_transition_matrix_fast(train, ALL_ACTIONS_LIST)
print("Transition matrix shape:", TRANS_MATRIX_LOG.shape)

All actions: ['allogroom', 'approach', 'attack', 'attemptmount', 'avoid', 'biteobject', 'chase', 'chaseattack', 'climb', 'defend', 'dig', 'disengage', 'dominance', 'dominancegroom', 'dominancemount', 'ejaculate', 'escape', 'exploreobject', 'flinch', 'follow', 'freeze', 'genitalgroom', 'huddle', 'intromit', 'mount', 'rear', 'reciprocalsniff', 'rest', 'run', 'selfgroom', 'shepherd', 'sniff', 'sniffbody', 'sniffface', 'sniffgenital', 'submit', 'tussle', 'background']
Computing transition matrix from metadata...


100%|██████████| 8789/8789 [00:01<00:00, 7483.44it/s] 

Transition matrix shape: (38, 38)


In [7]:
def create_solution_df(dataset):
    """Create the solution dataframe for validating out-of-fold predictions.

    From https://www.kaggle.com/code/ambrosm/mabe-validated-baseline-without-machine-learning/
    
    Parameters:
    dataset: (a subset of) the train dataframe
    
    Return values:
    solution: solution dataframe in the correct format for the score() function
    """
    solution = []
    for _, row in tqdm(dataset.iterrows(), total=len(dataset)):
    
        # Load annotation file
        lab_id = row['lab_id']
        if lab_id.startswith('MABe22'): continue
        video_id = row['video_id']
        path = f"/kaggle/input/MABe-mouse-behavior-detection/train_annotation/{lab_id}/{video_id}.parquet"
        try:
            annot = pd.read_parquet(path)
        except FileNotFoundError:
            # MABe22 and one more training file lack annotations.
            if verbose: print(f"No annotations for {path}")
            continue
    
        # Add all annotations to the solution
        annot['lab_id'] = lab_id
        annot['video_id'] = video_id
        annot['behaviors_labeled'] = row['behaviors_labeled']
        annot['target_id'] = np.where(annot.target_id != annot.agent_id, annot['target_id'].apply(lambda s: f"mouse{s}"), 'self')
        annot['agent_id'] = annot['agent_id'].apply(lambda s: f"mouse{s}")
        solution.append(annot)
    
    solution = pd.concat(solution)
    return solution

if validate_or_submit == 'validate':
    solution = create_solution_df(train_without_mabe22)

In [8]:
if validate_or_submit == 'stresstest':
    n_videos_per_lab = 2
    
    try:
        os.mkdir(f"stresstest_tracking")
    except FileExistsError:
        pass
    
    rng = np.random.default_rng()
    stresstest = pd.concat(
        [train.query("video_id == 1459695188")] # long video from BoisterousParrot
        + [df.sample(min(n_videos_per_lab, len(df)), random_state=1) for (_, df) in train.groupby('lab_id')])
    for _, row in tqdm(stresstest.iterrows(), total=len(stresstest)):
        lab_id = row['lab_id']
        video_id = row['video_id']
        
        # Load video
        path = f"/kaggle/input/MABe-mouse-behavior-detection/train_tracking/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
    
        if video_id == 1459695188: # long video from BoisterousParrot
            vid = pd.concat([vid] * 3) # provoke out of memory (5 is too much)
            vid['video_frame'] = np.arange(len(vid))
    
        # Drop some complete frames
        dropped_frames = list(rng.choice(np.unique(vid.video_frame), size=100, replace=False))
        vid = vid.query("~ video_frame.isin(@dropped_frames)")
        
        # Drop a complete bodypart
        if rng.uniform() < 0.2:
            dropped_bodypart = rng.choice(np.unique(vid.bodypart), size=1, replace=False)[0]
            vid = vid.query("bodypart != @dropped_bodypart")
        
        # Drop a mouse
        if rng.uniform() < 0.1:
            vid = vid.query("mouse_id != 1")
        
        # Drop random bodyparts from random frames
        if rng.uniform() < 0.7:
            mask = np.ones(len(vid), dtype=bool)
            mask[:int(0.4 * len(mask))] = False
            rng.shuffle(mask)
            vid = vid[mask]
    
        # Set random coordinates of bodyparts to nan
        if rng.uniform() < 0.7:
            mask = np.ones(len(vid), dtype=bool)
            mask[:int(0.2 * len(mask))] = False
            rng.shuffle(mask)
            vid.loc[:, 'x'] = np.where(mask, np.nan, vid.loc[:, 'x'])
            rng.shuffle(mask)
            vid.loc[:, 'y'] = np.where(mask, np.nan, vid.loc[:, 'y'])
    
        # Save the video
        try:
            os.mkdir(f"stresstest_tracking/{lab_id}")
        except FileExistsError:
            pass
        new_path = f"stresstest_tracking/{lab_id}/{video_id}.parquet"
        vid.to_parquet(new_path)


In [9]:
drop_body_parts =  ['headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 
                    'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 
                    'spine_1', 'spine_2',
                    'tail_middle_1', 'tail_middle_2', 'tail_midpoint']

def _fps_from_meta(meta_df, fallback_lookup, default_fps=30.0):
    """Extract FPS from metadata with fallback options"""
    if 'frames_per_second' in meta_df.columns and pd.notnull(meta_df['frames_per_second']).any():
        return float(meta_df['frames_per_second'].iloc[0])
    if 'fps' in meta_df.columns and pd.notnull(meta_df['fps']).any():
        return float(meta_df['fps'].iloc[0])
    vid = meta_df['video_id'].iloc[0]
    return float(fallback_lookup.get(vid, default_fps))

def generate_mouse_data(dataset, traintest, traintest_directory=None, generate_single=True, generate_pair=True):
    """Generate batches of data in coordinate representation (FPS-Aware).

    The batches have variable length, and every batch can have other columns
    for the labels, depending on what behaviors
    were labeled for the batch.

    Every video can produce zero, one or two batches.
    
    Parameters
    ----------
    dataset: (subset of) train.csv or test.csv dataframe
    traintest: either 'train' or 'test'

    Yields
    ------
    switch: either 'single' or 'pair'
    data: dataframe containing coordinates of the body parts of a single mouse or of a pair of mice
    meta: dataframe with columns ['video_id', 'agent_id', 'target_id', 'video_frame', 'frames_per_second']
    label: dataframe with labels (0, 1), one column per action, only if traintest == 'train'
    actions: list of actions to be predicted for this batch, only if traintest == 'test'
    """
    assert traintest in ['train', 'test']
    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"
    for _, row in dataset.iterrows():
        
        # Load the video and pivot it so that one frame = one row
        lab_id = row.lab_id
        if lab_id.startswith('MABe22'): continue
        video_id = row.video_id

        if type(row.behaviors_labeled) != str:
            # We cannot use videos without labeled behaviors
            if verbose: print('No labeled behaviors:', lab_id, video_id, type(row.behaviors_labeled))
            continue

        # Get FPS for this video
        fps = row.get('frames_per_second', row.get('fps_approx', 30.0))

        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
        if len(np.unique(vid.bodypart)) > 5:
            vid = vid.query("~ bodypart.isin(@drop_body_parts)")
        pvid = vid.pivot(columns=['mouse_id', 'bodypart'], index='video_frame', values=['x', 'y'])
        if pvid.isna().any().any():
            if verbose and traintest == 'test': print('video with missing values', video_id, traintest, len(vid), 'frames')
        else:
            if verbose and traintest == 'test': print('video with all values', video_id, traintest, len(vid), 'frames')
        del vid
        pvid = pvid.reorder_levels([1, 2, 0], axis=1).T.sort_index().T # mouse_id, body_part, xy
        pvid /= row.pix_per_cm_approx # convert to cm

        # Determine the behaviors of this video
        vid_behaviors = json.loads(row.behaviors_labeled)
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])
        
        # Load the annotations
        if traintest == 'train':
            try:
                annot = pd.read_parquet(path.replace('train_tracking', 'train_annotation'))
            except FileNotFoundError:
                # MABe22 and one more training file lack annotations.
                # We simply drop these videos.
                continue

        # Create the single_mouse dataframes: single_mouse, single_mouse_label and single_mouse_meta
        if generate_single:
            vid_behaviors_subset = vid_behaviors.query("target == 'self'") # single-mouse behaviors of this video
            for mouse_id_str in np.unique(vid_behaviors_subset.agent):
                try:
                    mouse_id = int(mouse_id_str[-1])
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("agent == @mouse_id_str").action)
                    single_mouse = pvid.loc[:, mouse_id]
                    assert len(single_mouse) == len(pvid)
                    single_mouse_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': mouse_id_str,
                        'target_id': 'self',
                        'video_frame': single_mouse.index,
                        'frames_per_second': fps
                    })
                    if traintest == 'train':
                        single_mouse_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=single_mouse.index)
                        annot_subset = annot.query("(agent_id == @mouse_id) & (target_id == @mouse_id)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            single_mouse_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'single', single_mouse, single_mouse_meta, single_mouse_label
                    else:
                        if verbose: print('- test single', video_id, mouse_id)
                        yield 'single', single_mouse, single_mouse_meta, vid_agent_actions
                except KeyError:
                    pass # If there is no data for the selected agent mouse, we skip the mouse.

        # Create the mouse_pair dataframes: mouse_pair, mouse_label and mouse_meta
        if generate_pair:
            vid_behaviors_subset = vid_behaviors.query("target != 'self'")
            if len(vid_behaviors_subset) > 0:
                for agent, target in itertools.permutations(np.unique(pvid.columns.get_level_values('mouse_id')), 2): # int8
                    agent_str = f"mouse{agent}"
                    target_str = f"mouse{target}"
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("(agent == @agent_str) & (target == @target_str)").action)
                    mouse_pair = pd.concat([pvid[agent], pvid[target]], axis=1, keys=['A', 'B'])
                    assert len(mouse_pair) == len(pvid)
                    mouse_pair_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': agent_str,
                        'target_id': target_str,
                        'video_frame': mouse_pair.index,
                        'frames_per_second': fps
                    })
                    if traintest == 'train':
                        mouse_pair_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=mouse_pair.index)
                        annot_subset = annot.query("(agent_id == @agent) & (target_id == @target)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            mouse_pair_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'pair', mouse_pair, mouse_pair_meta, mouse_pair_label
                    else:
                        if verbose: print('- test pair', video_id, agent, target)
                        yield 'pair', mouse_pair, mouse_pair_meta, vid_agent_actions

In [10]:
# =============================================================================
# CELL: ACTION-SPECIFIC THRESHOLDS (From FPS-Aware v4 Reference Notebook)
# =============================================================================
from collections import defaultdict

# Action-specific threshold configuration (from v4)
# Structure supports mode-aware thresholds (single vs pair behaviors)
# 
# How to tune thresholds:
# 1. Set "default" for global fallback (used if mode defaults not specified)
# 2. Set "single_default" and "pair_default" for mode-specific defaults
# 3. Add action-specific overrides in "single" or "pair" dictionaries
# 4. Actions not explicitly listed will use the appropriate mode default
#
# Threshold range: typically 0.20-0.40, with 0.27 as a good starting point
# Higher thresholds = fewer false positives, more false negatives
# Lower thresholds = more false positives, fewer false negatives

action_thresholds = {
    "default": 0.27,           # Global fallback threshold
    "single_default": 0.26,    # Default for single mouse behaviors (target_id == 'self')
    "pair_default": 0.28,      # Default for pair behaviors (target_id != 'self')
    "single": {
        "rear": 0.30,          # Higher threshold - distinctive behavior, reduce false positives
        "groom": 0.28,         # Slightly higher - common behavior, needs good confidence
        "sniff": 0.25,         # Lower threshold - subtle behavior, improve recall
        "dig": 0.29,           # Higher threshold - distinctive behavior
        "eat": 0.27,           # Standard threshold - balanced precision/recall
        "drink": 0.27,         # Standard threshold - balanced precision/recall
        "sleep": 0.24,         # Lower threshold - rare but important, improve recall
    },
    "pair": {
        "attack": 0.24,        # Lower threshold - rare but critical behavior, maximize recall
        "mount": 0.28,         # Higher threshold - distinctive behavior, reduce false positives
        "sniff": 0.26,         # Lower threshold - subtle social behavior, improve recall
        "groom": 0.27,         # Standard threshold - balanced precision/recall
        "chase": 0.25,         # Lower threshold - important social behavior, improve recall
        "follow": 0.26,        # Lower threshold - subtle behavior, improve recall
        "approach": 0.27,      # Standard threshold - balanced precision/recall
    },
}

def _select_threshold_map(thresholds, mode: str):
    """Select the correct threshold map based on mode (single vs pair)"""
    if isinstance(thresholds, dict):
        # mode-aware?
        if ("single" in thresholds) or ("pair" in thresholds) or \
           ("single_default" in thresholds) or ("pair_default" in thresholds):
            base_default = float(thresholds.get("default", 0.27))
            mode_default = float(thresholds.get(f"{mode}_default", base_default))
            mode_overrides = thresholds.get(mode, {}) or {}
            out = defaultdict(lambda: mode_default)
            out.update({str(k): float(v) for k, v in mode_overrides.items()})
            return out
        # plain per-action dict
        out = defaultdict(lambda: float(thresholds.get("default", 0.27)))
        out.update({str(k): float(v) for k, v in thresholds.items() if k != "default"})
        return out
    return defaultdict(lambda: 0.27)

In [11]:
# =============================================================================
# CELL: PREDICT MULTICLASS (FPS-Aware Adaptive Thresholding from v4)
# =============================================================================
def predict_multiclass_adaptive(pred, meta, action_thresholds):
    """Adaptive thresholding per action + FPS-aware temporal smoothing"""
    # Extract FPS for FPS-aware smoothing and filtering
    fps = _fps_from_meta(meta, {}, default_fps=30.0)
    
    # Apply FPS-aware temporal smoothing (scaled to configurable base window)
    smoothing_window = max(3, int(round(SMOOTHING_WINDOW_FRAMES_AT_30FPS * fps / 30.0)))
    pred_smoothed = pred.rolling(window=smoothing_window, min_periods=1, center=True).mean()

    # Robust mode detection: check if all target_ids are 'self' for single mode
    mode = 'pair'  # default to pair
    try:
        if 'target_id' in meta.columns:
            unique_targets = meta['target_id'].unique()
            if len(unique_targets) == 1 and unique_targets[0] == 'self':
                mode = 'single'
    except (KeyError, AttributeError, Exception):
        # If detection fails, default to pair mode
        pass

    ama = np.argmax(pred_smoothed, axis=1)
    th_map = _select_threshold_map(action_thresholds, mode)

    max_probs = pred_smoothed.max(axis=1)
    threshold_mask = np.zeros(len(pred_smoothed), dtype=bool)
    for i, action in enumerate(pred_smoothed.columns):
        action_mask = (ama == i)
        threshold = th_map[action]
        threshold_mask |= (action_mask & (max_probs >= threshold))

    ama = np.where(threshold_mask, ama, -1)
    ama = pd.Series(ama, index=meta.video_frame)
    
    changes_mask = (ama != ama.shift(1)).values
    ama_changes = ama[changes_mask]
    meta_changes = meta[changes_mask]
    mask = ama_changes.values >= 0
    mask[-1] = False
    
    submission_part = pd.DataFrame({
        'video_id': meta_changes['video_id'][mask].values,
        'agent_id': meta_changes['agent_id'][mask].values,
        'target_id': meta_changes['target_id'][mask].values,
        'action': pred.columns[ama_changes[mask].values],
        'start_frame': ama_changes.index[mask],
        'stop_frame': ama_changes.index[1:][mask[:-1]]
    })
    
    # Handle stop frames: if next change is in different video/agent/target, set stop to end of current video
    stop_meta = meta_changes.iloc[1:][mask[:-1]] if len(mask) > 1 else pd.DataFrame()
    
    for i in range(len(submission_part)):
        video_id = submission_part.video_id.iloc[i]
        agent_id = submission_part.agent_id.iloc[i]
        target_id = submission_part.target_id.iloc[i]
        
        # Check if we need to adjust stop frame
        if i < len(stop_meta):
            next_row = stop_meta.iloc[i]
            if (next_row['video_id'] != video_id or 
                next_row['agent_id'] != agent_id or 
                next_row['target_id'] != target_id):
                # Next event is different video/agent/target, set stop to end of current video
                new_stop_frame = meta.query("(video_id == @video_id)").video_frame.max() + 1
                submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame
        else:
            # Last event, set stop to end of video
            new_stop_frame = meta.query("(video_id == @video_id)").video_frame.max() + 1
            submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame
    
    # Filter out very short events (likely noise) - FPS-aware minimum duration
    min_duration_frames = max(2, int(round(MIN_DURATION_SECONDS * fps)))
    duration = submission_part.stop_frame - submission_part.start_frame
    submission_part = submission_part[duration >= min_duration_frames].reset_index(drop=True)
    
    if len(submission_part) > 0:
        assert (submission_part.stop_frame > submission_part.start_frame).all(), 'stop <= start'
    
    if verbose: print(f'  actions found: {len(submission_part)}')
    return submission_part

In [12]:
# =============================================================================
# CELL: ADVANCED FEATURE ENGINEERING (Updated with Interaction V2)
# =============================================================================
import numpy as np
import pandas as pd
import itertools

def _scale(n_frames_at_30fps, fps, ref=30.0):
    return max(1, int(round(n_frames_at_30fps * float(fps) / ref)))

def _scale_signed(n_frames_at_30fps, fps, ref=30.0):
    if n_frames_at_30fps == 0: return 0
    s = 1 if n_frames_at_30fps > 0 else -1
    mag = max(1, int(round(abs(n_frames_at_30fps) * float(fps) / ref)))
    return s * mag

# --- Feature Helpers ---
def add_curvature_features(X, center_x, center_y, fps):
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    acc_x = vel_x.diff()
    acc_y = vel_y.diff()
    cross_prod = vel_x * acc_y - vel_y * acc_x
    vel_mag = np.sqrt(vel_x**2 + vel_y**2)
    curvature = np.abs(cross_prod) / (vel_mag**3 + 1e-6)
    for w in [30, 60]:
        ws = _scale(w, fps)
        X[f'curv_mean_{w}'] = curvature.rolling(ws, min_periods=max(1, ws // 6)).mean()
    angle = np.arctan2(vel_y, vel_x)
    angle_change = np.abs(angle.diff())
    w = 30
    ws = _scale(w, fps)
    X[f'turn_rate_{w}'] = angle_change.rolling(ws, min_periods=max(1, ws // 6)).sum()
    return X

def add_multiscale_features(X, center_x, center_y, fps):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)
    scales = [10, 40, 160]
    for scale in scales:
        ws = _scale(scale, fps)
        if len(speed) >= ws:
            X[f'sp_m{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).mean()
            X[f'sp_s{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).std()
    if len(scales) >= 2 and f'sp_m{scales[0]}' in X.columns and f'sp_m{scales[-1]}' in X.columns:
        X['sp_ratio'] = X[f'sp_m{scales[0]}'] / (X[f'sp_m{scales[-1]}'] + 1e-6)
    return X

def add_state_features(X, center_x, center_y, fps):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)
    w_ma = _scale(15, fps)
    speed_ma = speed.rolling(w_ma, min_periods=max(1, w_ma // 3)).mean()
    try:
        bins = [-np.inf, 0.5 * fps, 2.0 * fps, 5.0 * fps, np.inf]
        speed_states = pd.cut(speed_ma, bins=bins, labels=[0, 1, 2, 3]).astype(float)
        for window in [60, 120]:
            ws = _scale(window, fps)
            if len(speed_states) >= ws:
                for state in [0, 1, 2, 3]:
                    X[f's{state}_{window}'] = ((speed_states == state).astype(float)
                        .rolling(ws, min_periods=max(1, ws // 6)).mean())
                state_changes = (speed_states != speed_states.shift(1)).astype(float)
                X[f'trans_{window}'] = state_changes.rolling(ws, min_periods=max(1, ws // 6)).sum()
    except: pass
    return X

def add_longrange_features(X, center_x, center_y, fps):
    for window in [120, 240]:
        ws = _scale(window, fps)
        if len(center_x) >= ws:
            X[f'x_ml{window}'] = center_x.rolling(ws, min_periods=max(5, ws // 6)).mean()
            X[f'y_ml{window}'] = center_y.rolling(ws, min_periods=max(5, ws // 6)).mean()
    for span in [60, 120]:
        s = _scale(span, fps)
        X[f'x_e{span}'] = center_x.ewm(span=s, min_periods=1).mean()
        X[f'y_e{span}'] = center_y.ewm(span=s, min_periods=1).mean()
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)
    for window in [60, 120]:
        ws = _scale(window, fps)
        if len(speed) >= ws:
            X[f'sp_pct{window}'] = speed.rolling(ws, min_periods=max(5, ws // 6)).rank(pct=True)
    return X

def add_interaction_features_v2(X, mouse_pair, avail_A, avail_B, fps):
    """NEW: Leader-Follower & Chase Dynamics features from Reference Notebook"""
    if 'body_center' not in avail_A or 'body_center' not in avail_B:
        return X
    
    rel_x = mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x']
    rel_y = mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y']
    rel_dist = np.sqrt(rel_x**2 + rel_y**2)
    
    A_vx = mouse_pair['A']['body_center']['x'].diff()
    A_vy = mouse_pair['A']['body_center']['y'].diff()
    B_vx = mouse_pair['B']['body_center']['x'].diff()
    B_vy = mouse_pair['B']['body_center']['y'].diff()
    
    # Leader score: Positive if moving TOWARDS other mouse
    A_lead = (A_vx * rel_x + A_vy * rel_y) / (np.sqrt(A_vx**2 + A_vy**2) * rel_dist + 1e-6)
    B_lead = (B_vx * (-rel_x) + B_vy * (-rel_y)) / (np.sqrt(B_vx**2 + B_vy**2) * rel_dist + 1e-6)
    
    for window in [30, 60]:
        ws = _scale(window, fps)
        X[f'A_ld{window}'] = A_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()
        X[f'B_ld{window}'] = B_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()
        
    # Chase metric
    approach = -rel_dist.diff()
    chase = approach * B_lead
    w = 30
    ws = _scale(w, fps)
    X[f'chase_{w}'] = chase.rolling(ws, min_periods=max(1, ws // 6)).mean()
    
    # Velocity Correlation
    for window in [60, 120]:
        ws = _scale(window, fps)
        A_sp = np.sqrt(A_vx**2 + A_vy**2)
        B_sp = np.sqrt(B_vx**2 + B_vy**2)
        X[f'sp_cor{window}'] = A_sp.rolling(ws, min_periods=max(1, ws // 6)).corr(B_sp)
        
    return X

# --- Main Transforms ---

def transform_single(single_mouse, body_parts_tracked, fps=30.0):
    available_body_parts = single_mouse.columns.get_level_values(0)
    X = pd.DataFrame({
        f"{p1}+{p2}": np.square(single_mouse[p1] - single_mouse[p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.combinations(body_parts_tracked, 2)
        if p1 in available_body_parts and p2 in available_body_parts
    })
    X = X.reindex(columns=[f"{p1}+{p2}" for p1, p2 in itertools.combinations(body_parts_tracked, 2)], copy=False)

    if all(p in single_mouse.columns for p in ['ear_left', 'ear_right', 'tail_base']):
        lag = _scale(10, fps)
        shifted = single_mouse[['ear_left', 'ear_right', 'tail_base']].shift(lag)
        speeds = pd.DataFrame({
            'sp_lf': np.square(single_mouse['ear_left'] - shifted['ear_left']).sum(axis=1, skipna=False),
            'sp_rt': np.square(single_mouse['ear_right'] - shifted['ear_right']).sum(axis=1, skipna=False),
            'sp_lf2': np.square(single_mouse['ear_left'] - shifted['tail_base']).sum(axis=1, skipna=False),
            'sp_rt2': np.square(single_mouse['ear_right'] - shifted['tail_base']).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elong'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)

    if all(p in available_body_parts for p in ['nose', 'body_center', 'tail_base']):
        v1 = single_mouse['nose'] - single_mouse['body_center']
        v2 = single_mouse['tail_base'] - single_mouse['body_center']
        X['body_ang'] = (v1['x'] * v2['x'] + v1['y'] * v2['y']) / (
            np.sqrt(v1['x']**2 + v1['y']**2) * np.sqrt(v2['x']**2 + v2['y']**2) + 1e-6)

    if 'body_center' in available_body_parts:
        cx = single_mouse['body_center']['x']
        cy = single_mouse['body_center']['y']
        for w in [5, 15, 30, 60]:
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'cx_m{w}'] = cx.rolling(ws, **roll).mean()
            X[f'cy_m{w}'] = cy.rolling(ws, **roll).mean()
            X[f'cx_s{w}'] = cx.rolling(ws, **roll).std()
            X[f'cy_s{w}'] = cy.rolling(ws, **roll).std()
            X[f'x_rng{w}'] = cx.rolling(ws, **roll).max() - cx.rolling(ws, **roll).min()
            X[f'y_rng{w}'] = cy.rolling(ws, **roll).max() - cy.rolling(ws, **roll).min()
            X[f'disp{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).sum()**2 +
                                     cy.diff().rolling(ws, min_periods=1).sum()**2)
            X[f'act{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).var() +
                                   cy.diff().rolling(ws, min_periods=1).var())

        X = add_curvature_features(X, cx, cy, fps)
        X = add_multiscale_features(X, cx, cy, fps)
        X = add_state_features(X, cx, cy, fps)
        X = add_longrange_features(X, cx, cy, fps)

    if all(p in available_body_parts for p in ['nose', 'tail_base']):
        nt_dist = np.sqrt((single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 +
                          (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nt_lg{lag}'] = nt_dist.shift(l)
            X[f'nt_df{lag}'] = nt_dist - nt_dist.shift(l)

    if all(p in available_body_parts for p in ['ear_left', 'ear_right']):
        ear_d = np.sqrt((single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 +
                        (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2)
        for off in [-30, -20, -10, 10, 20, 30]:
            o = _scale_signed(off, fps)
            X[f'ear_o{off}'] = ear_d.shift(-o)
        w = _scale(30, fps)
        X['ear_con'] = ear_d.rolling(w, min_periods=1, center=True).std() / \
                       (ear_d.rolling(w, min_periods=1, center=True).mean() + 1e-6)

    return X.astype(np.float32, copy=False)

def transform_pair(mouse_pair, body_parts_tracked, fps=30.0):
    avail_A = mouse_pair['A'].columns.get_level_values(0)
    avail_B = mouse_pair['B'].columns.get_level_values(0)

    X = pd.DataFrame({
        f"12+{p1}+{p2}": np.square(mouse_pair['A'][p1] - mouse_pair['B'][p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.product(body_parts_tracked, repeat=2)
        if p1 in avail_A and p2 in avail_B
    })
    X = X.reindex(columns=[f"12+{p1}+{p2}" for p1, p2 in itertools.product(body_parts_tracked, repeat=2)], copy=False)

    if ('A', 'ear_left') in mouse_pair.columns and ('B', 'ear_left') in mouse_pair.columns:
        lag = _scale(10, fps)
        shA = mouse_pair['A']['ear_left'].shift(lag)
        shB = mouse_pair['B']['ear_left'].shift(lag)
        speeds = pd.DataFrame({
            'sp_A': np.square(mouse_pair['A']['ear_left'] - shA).sum(axis=1, skipna=False),
            'sp_AB': np.square(mouse_pair['A']['ear_left'] - shB).sum(axis=1, skipna=False),
            'sp_B': np.square(mouse_pair['B']['ear_left'] - shB).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elong'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)

    if all(p in avail_A for p in ['nose', 'tail_base']) and all(p in avail_B for p in ['nose', 'tail_base']):
        dir_A = mouse_pair['A']['nose'] - mouse_pair['A']['tail_base']
        dir_B = mouse_pair['B']['nose'] - mouse_pair['B']['tail_base']
        X['rel_ori'] = (dir_A['x'] * dir_B['x'] + dir_A['y'] * dir_B['y']) / (
            np.sqrt(dir_A['x']**2 + dir_A['y']**2) * np.sqrt(dir_B['x']**2 + dir_B['y']**2) + 1e-6)

    if all(p in avail_A for p in ['nose']) and all(p in avail_B for p in ['nose']):
        cur = np.square(mouse_pair['A']['nose'] - mouse_pair['B']['nose']).sum(axis=1, skipna=False)
        lag = _scale(10, fps)
        shA_n = mouse_pair['A']['nose'].shift(lag)
        shB_n = mouse_pair['B']['nose'].shift(lag)
        past = np.square(shA_n - shB_n).sum(axis=1, skipna=False)
        X['appr'] = cur - past

    if 'body_center' in avail_A and 'body_center' in avail_B:
        cd = np.sqrt((mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x'])**2 +
                     (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y'])**2)
        X['v_cls'] = (cd < 5.0).astype(float)
        X['cls']   = ((cd >= 5.0) & (cd < 15.0)).astype(float)
        X['med']   = ((cd >= 15.0) & (cd < 30.0)).astype(float)
        X['far']   = (cd >= 30.0).astype(float)
        
        # --- ORIGINAL INTERACTION FEATURES ---
        cd_full = np.square(mouse_pair['A']['body_center'] - mouse_pair['B']['body_center']).sum(axis=1, skipna=False)
        for w in [5, 15, 30, 60]:
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'd_m{w}']  = cd_full.rolling(ws, **roll).mean()
            X[f'd_s{w}']  = cd_full.rolling(ws, **roll).std()
            X[f'd_mn{w}'] = cd_full.rolling(ws, **roll).min()
            X[f'd_mx{w}'] = cd_full.rolling(ws, **roll).max()

            d_var = cd_full.rolling(ws, **roll).var()
            X[f'int{w}'] = 1 / (1 + d_var)

            Axd = mouse_pair['A']['body_center']['x'].diff()
            Ayd = mouse_pair['A']['body_center']['y'].diff()
            Bxd = mouse_pair['B']['body_center']['x'].diff()
            Byd = mouse_pair['B']['body_center']['y'].diff()
            coord = Axd * Bxd + Ayd * Byd
            X[f'co_m{w}'] = coord.rolling(ws, **roll).mean()
            X[f'co_s{w}'] = coord.rolling(ws, **roll).std()

        # --- USE THE NEW INTERACTION V2 FUNCTION HERE ---
        X = add_interaction_features_v2(X, mouse_pair, avail_A, avail_B, fps)

    if 'nose' in avail_A and 'nose' in avail_B:
        nn = np.sqrt((mouse_pair['A']['nose']['x'] - mouse_pair['B']['nose']['x'])**2 +
                     (mouse_pair['A']['nose']['y'] - mouse_pair['B']['nose']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nn_lg{lag}']  = nn.shift(l)
            X[f'nn_ch{lag}']  = nn - nn.shift(l)
            is_cl = (nn < 10.0).astype(float)
            X[f'cl_ps{lag}']  = is_cl.rolling(l, min_periods=1).mean()

    if 'body_center' in avail_A and 'body_center' in avail_B:
        Avx = mouse_pair['A']['body_center']['x'].diff()
        Avy = mouse_pair['A']['body_center']['y'].diff()
        Bvx = mouse_pair['B']['body_center']['x'].diff()
        Bvy = mouse_pair['B']['body_center']['y'].diff()
        val = (Avx * Bvx + Avy * Bvy) / (np.sqrt(Avx**2 + Avy**2) * np.sqrt(Bvx**2 + Bvy**2) + 1e-6)

        for off in [-20, -10, 0, 10, 20]:
            o = _scale_signed(off, fps)
            X[f'va_{off}'] = val.shift(-o)

        w = _scale(30, fps)
        X['int_con'] = cd_full.rolling(w, min_periods=1, center=True).std() / \
                       (cd_full.rolling(w, min_periods=1, center=True).mean() + 1e-6)

    return X.astype(np.float32, copy=False)

In [13]:
# =============================================================================
# CELL: Enhanced cross_validate_classifier (Replace in cross_validate_classifier)
# =============================================================================

# ADD this at the beginning of cross_validate_classifier function,
# right after the function definition line:

def cross_validate_classifier(binary_classifier, X, label, meta):
    """Cross-validate a binary classifier per action and a multi-class classifier over all actions.

    Parameters
    ----------
    binary_classifier: classifier with predict_proba (can be a list of classifiers for ensemble)
    X: 2d array-like (distance representation) of shape (n_samples, n_features)
    label: dataframe with binary targets (one column per action, may have missing values), index doesn't matter
    meta: dataframe with columns ['video_id', 'agent_id', 'target_id', 'video_frame'], index doesn't matter

    Output
    ------
    appends to f1_list (binary) and submission_list (multi-class)
    
    """
    # Cross-validate a binary classifier for every action
    oof = pd.DataFrame(index=meta.video_frame) # will get a column per action
    for action in label.columns:
        # Filter for samples (video frames) with a defined target (i.e., target is not nan)
        action_mask = ~ label[action].isna().values
        X_action = X[action_mask]
        y_action = label[action][action_mask].values.astype(int)
        p = y_action.mean()
        baseline_score = p / (1 + p)
        groups_action = meta.video_id[action_mask] # ensure validation has unseen videos
        if len(np.unique(groups_action)) < 5:
            continue # GroupKFold would fail with fewer than n_splits groups

        if not (y_action == 0).all():
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore', category=RuntimeWarning)
                # Use first model from ensemble for cross-validation
                if isinstance(binary_classifier, list):
                    oof_action = cross_val_predict(binary_classifier[0], X_action, y_action, 
                                                   groups=groups_action, cv=GroupKFold(), method='predict_proba')
                else:
                    oof_action = cross_val_predict(binary_classifier, X_action, y_action, 
                                                   groups=groups_action, cv=GroupKFold(), method='predict_proba')
            oof_action = oof_action[:, 1]
        else:
            oof_action = np.zeros(len(y_action))
        
        # Use adaptive threshold for this action
        mode = 'single' if 'target_id' in meta.columns and meta['target_id'].eq('self').all() else 'pair'
        th_map = _select_threshold_map(action_thresholds, mode)
        action_threshold = th_map[action]
        
        f1 = f1_score(y_action, (oof_action >= action_threshold), zero_division=0)
        ch = '>' if f1 > baseline_score else '=' if f1 == baseline_score else '<'
        print(f"  F1: {f1:.3f} {ch} ({baseline_score:.3f}) {action} (th={action_threshold:.3f})")
        f1_list.append((body_parts_tracked_str, action, f1))
        oof_column = np.zeros(len(label))
        oof_column[action_mask] = oof_action
        oof[action] = oof_column

    # Make the multi-class prediction with adaptive thresholding
    submission_part = predict_multiclass_adaptive(oof, meta, action_thresholds)
    submission_list.append(submission_part)

In [14]:
# =============================================================================
# CELL: SUBMIT ENSEMBLE (FPS-Aware Version from v4)
# =============================================================================
def submit_ensemble(body_parts_tracked_str, switch_tr, X_tr, label, n_samples):
    # Train ensemble of models for every action
    X_tr_np = X_tr.to_numpy(np.float32, copy=False) if hasattr(X_tr, 'to_numpy') else np.asarray(X_tr, dtype=np.float32)
    del X_tr; gc.collect()
    
    # --- MODEL DEFINITIONS ---
    models = []
    
    # Model 1: LGBM 225 (Baseline from v4)
    models.append(make_pipeline(
        StratifiedSubsetClassifier(
            lightgbm.LGBMClassifier(
                n_estimators=225, learning_rate=0.07, min_child_samples=40,
                num_leaves=31, subsample=0.8, colsample_bytree=0.8, verbose=-1,
                random_state=SEED, bagging_seed=SEED, feature_fraction_seed=SEED, data_random_seed=SEED
            ), int(n_samples/1.3),
        )
    ))
    
    # Model 2: LGBM 150 (Deep)
    models.append(make_pipeline(
        StratifiedSubsetClassifier(
            lightgbm.LGBMClassifier(
                n_estimators=150, learning_rate=0.1, min_child_samples=20,
                num_leaves=63, max_depth=8, subsample=0.7, colsample_bytree=0.9,
                reg_alpha=0.1, reg_lambda=0.1, verbose=-1,
                random_state=SEED, bagging_seed=SEED, feature_fraction_seed=SEED, data_random_seed=SEED
            ), int(n_samples/2),
        )
    ))
    
    # Model 3: LGBM 100 (Very Deep)
    models.append(make_pipeline(
        StratifiedSubsetClassifier(
            lightgbm.LGBMClassifier(
                n_estimators=100, learning_rate=0.05, min_child_samples=30,
                num_leaves=127, max_depth=10, subsample=0.75, verbose=-1,
                random_state=SEED, bagging_seed=SEED, feature_fraction_seed=SEED, data_random_seed=SEED
            ), int(n_samples/3),
        )
    ))
    
    if 'XGBOOST_AVAILABLE' in globals() and XGBOOST_AVAILABLE:
        models.append(make_pipeline(
            StratifiedSubsetClassifier(
                XGBClassifier(
                    n_estimators=180, learning_rate=0.08, max_depth=6,
                    min_child_weight=5, subsample=0.8, colsample_bytree=0.8,
                    tree_method='hist', verbosity=0,
                    random_state=SEED
                ), int(n_samples/1.5),
            )
        ))
    
    if 'CATBOOST_AVAILABLE' in globals() and CATBOOST_AVAILABLE:
        models.append(make_pipeline(
            StratifiedSubsetClassifier(
                CatBoostClassifier(
                    iterations=120, learning_rate=0.1, depth=6,
                    verbose=False, allow_writing_files=False,
                    random_seed=SEED
                ), n_samples,
            )
        ))

    # --- TRAINING LOOP ---
    model_list = []
    for action in label.columns:
        y_raw = label[action].to_numpy()
        mask = ~pd.isna(y_raw)
        y_action = y_raw[mask].astype(int)
        if not (y_action == 0).all() and np.sum(y_action) >= 5:
            trained = []
            idx = np.flatnonzero(mask)
            for m in models:
                m_clone = clone(m)
                m_clone.fit(X_tr_np[idx], y_action)
                trained.append(m_clone)
            model_list.append((action, trained))

    del X_tr_np; gc.collect()

    # --- PREDICTION LOOP ---
    body_parts_tracked = json.loads(body_parts_tracked_str)
    if len(body_parts_tracked) > 5:
        body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]

    test_subset = test[test.body_parts_tracked == body_parts_tracked_str]
    generator = generate_mouse_data(
        test_subset, 'test',
        generate_single=(switch_tr == 'single'),
        generate_pair=(switch_tr == 'pair')
    )

    fps_lookup = (
        test_subset[['video_id', 'frames_per_second']]
        .drop_duplicates('video_id')
        .set_index('video_id')['frames_per_second']
        .to_dict()
    )

    if verbose:
        print(f"n_videos: {len(test_subset)}, n_models: {len(models)}")

    for switch_te, data_te, meta_te, actions_te in generator:
        assert switch_te == switch_tr
        try:
            fps_i = _fps_from_meta(meta_te, fps_lookup, default_fps=30.0)

            if switch_te == 'single':
                X_te = transform_single(data_te, body_parts_tracked, fps_i).astype(np.float32)
            else:
                X_te = transform_pair(data_te, body_parts_tracked, fps_i).astype(np.float32)

            X_te_np = X_te.to_numpy(np.float32, copy=False)
            del X_te, data_te; gc.collect()

            pred = pd.DataFrame(index=meta_te.video_frame)
            for action, trained in model_list:
                if action in actions_te:
                    probs = [m.predict_proba(X_te_np)[:, 1] for m in trained]
                    pred[action] = np.mean(probs, axis=0)

            del X_te_np; gc.collect()

            if pred.shape[1] != 0:
                sub_part = predict_multiclass_adaptive(pred, meta_te, action_thresholds)
                submission_list.append(sub_part)
            else:
                if verbose:
                    print("  ERROR: no training data")

        except Exception as e:
            if verbose:
                error_msg = str(e)
                if len(error_msg) > 200:
                    error_msg = error_msg[:200] + "..."
                print(f"  ERROR: {error_msg}")
                if verbose:
                    print(f"  Traceback: {traceback.format_exc()[:300]}")
            try:
                del data_te
            except Exception:
                pass
            gc.collect()

In [15]:
# =============================================================================
# CELL: MAIN TRAINING LOOP (FPS-Aware Version from v4)
# =============================================================================
# %%time
f1_list = []
submission_list = []

print(f"XGBoost: {'XGBOOST_AVAILABLE' in globals() and XGBOOST_AVAILABLE}")
print(f"CatBoost: {'CATBOOST_AVAILABLE' in globals() and CATBOOST_AVAILABLE}\n")

for section in range(1, len(body_parts_tracked_list)):
    body_parts_tracked_str = body_parts_tracked_list[section]
    try:
        body_parts_tracked = json.loads(body_parts_tracked_str)
        print(f"{section}. Processing: {len(body_parts_tracked)} body parts")
        if len(body_parts_tracked) > 5:
            body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]

        train_subset = train[train.body_parts_tracked == body_parts_tracked_str]

        _fps_lookup = (
            train_subset[['video_id', 'frames_per_second']]
            .drop_duplicates('video_id')
            .set_index('video_id')['frames_per_second']
            .to_dict()
        )

        single_list, single_label_list, single_meta_list = [], [], []
        pair_list, pair_label_list, pair_meta_list = [], [], []

        for switch, data, meta, label in generate_mouse_data(train_subset, 'train'):
            if switch == 'single':
                single_list.append(data)
                single_meta_list.append(meta)
                single_label_list.append(label)
            else:
                pair_list.append(data)
                pair_meta_list.append(meta)
                pair_label_list.append(label)

        if len(single_list) > 0:
            single_feats_parts = []
            for data_i, meta_i in zip(single_list, single_meta_list):
                fps_i = _fps_from_meta(meta_i, _fps_lookup, default_fps=30.0)
                Xi = transform_single(data_i, body_parts_tracked, fps_i).astype(np.float32)
                single_feats_parts.append(Xi)

            X_tr = pd.concat(single_feats_parts, axis=0, ignore_index=True)
 
            single_label = pd.concat(single_label_list, axis=0, ignore_index=True)
            single_meta  = pd.concat(single_meta_list,  axis=0, ignore_index=True)

            del single_list, single_label_list, single_meta_list, single_feats_parts
            gc.collect()

            print(f"  Single: {X_tr.shape}")
            submit_ensemble(body_parts_tracked_str, 'single', X_tr, single_label, 2_000_000)

            del X_tr, single_label, single_meta
            gc.collect()

        if len(pair_list) > 0:
            pair_feats_parts = []
            for data_i, meta_i in zip(pair_list, pair_meta_list):
                fps_i = _fps_from_meta(meta_i, _fps_lookup, default_fps=30.0)
                Xi = transform_pair(data_i, body_parts_tracked, fps_i).astype(np.float32)
                pair_feats_parts.append(Xi)

            X_tr = pd.concat(pair_feats_parts, axis=0, ignore_index=True)

            
            pair_label = pd.concat(pair_label_list, axis=0, ignore_index=True)
            pair_meta  = pd.concat(pair_meta_list,  axis=0, ignore_index=True)

            del pair_list, pair_label_list, pair_meta_list, pair_feats_parts
            gc.collect()

            print(f"  Pair: {X_tr.shape}")
            submit_ensemble(body_parts_tracked_str, 'pair', X_tr, pair_label, 900_000)

            del X_tr, pair_label, pair_meta
            gc.collect()

    except Exception as e:
        error_msg = str(e)
        if len(error_msg) > 200:
            error_msg = error_msg[:200] + "..."
        print(f'***Exception*** {error_msg}')
        if verbose:
            print(f'Traceback: {traceback.format_exc()[:500]}')

    gc.collect()
    print()

XGBoost: False
CatBoost: True

1. Processing: 18 body parts
  Single: (544859, 117)
n_videos: 1, n_models: 4
video with missing values 438887472 test 529471 frames
- test single 438887472 1
  actions found: 8
- test single 438887472 2
  actions found: 73
- test single 438887472 3
  actions found: 50
- test single 438887472 4
  actions found: 123
  Pair: (1744248, 140)
n_videos: 1, n_models: 4
video with missing values 438887472 test 529471 frames
- test pair 438887472 1 2
  actions found: 0
- test pair 438887472 1 3
  actions found: 0
- test pair 438887472 1 4
  actions found: 10
- test pair 438887472 2 1
  actions found: 9
- test pair 438887472 2 3
  actions found: 21
- test pair 438887472 2 4
  actions found: 25
- test pair 438887472 3 1
  actions found: 13
- test pair 438887472 3 2
  actions found: 18
- test pair 438887472 3 4
  actions found: 21
- test pair 438887472 4 1
  actions found: 50
- test pair 438887472 4 2
  actions found: 41
- test pair 438887472 4 3
  actions found: 39


In [16]:
def robustify(submission, dataset, traintest, traintest_directory=None):
    """Ensure that the submission conforms to the three rules (with security fix from v4)"""
    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"

    # Rule 1: Ensure that start_frame < stop_frame
    submission = submission[submission.start_frame < submission.stop_frame]
    
    # Rule 2: Avoid multiple predictions for the same frame from one agent/target pair
    group_list = []
    for _, group in submission.groupby(['video_id', 'agent_id', 'target_id']):
        group = group.sort_values('start_frame')
        mask = np.ones(len(group), dtype=bool)
        last_stop = 0
        for i, (_, row) in enumerate(group.iterrows()):
            if row['start_frame'] < last_stop:
                mask[i] = False
            else:
                last_stop = row['stop_frame']
        group_list.append(group[mask])
    submission = pd.concat(group_list) if group_list else submission

    # Rule 3: Submit something for every video
    s_list = []
    for _, row in dataset.iterrows():
        lab_id = row['lab_id']
        video_id = row['video_id']
        if (submission.video_id == video_id).any():
            continue

        if verbose:
            print(f"Video {video_id} has no predictions")

        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)

        # SECURITY FIX: Use json.loads instead of eval or ast.literal_eval
        vid_behaviors = json.loads(row['behaviors_labeled'])
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])

        start_frame = vid.video_frame.min()
        stop_frame = vid.video_frame.max() + 1

        for (agent, target), actions in vid_behaviors.groupby(['agent', 'target']):
            batch_len = int(np.ceil((stop_frame - start_frame) / len(actions)))
            for i, (_, action_row) in enumerate(actions.iterrows()):
                batch_start = start_frame + i * batch_len
                batch_stop = min(batch_start + batch_len, stop_frame)
                s_list.append((video_id, agent, target, action_row['action'], batch_start, batch_stop))

    if len(s_list) > 0:
        submission = pd.concat([
            submission,
            pd.DataFrame(s_list, columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
        ])

    submission = submission.reset_index(drop=True)
    return submission

In [17]:
if validate_or_submit == 'validate':
    # Score the oof predictions with the competition scoring function
    submission = pd.concat(submission_list)
    submission_robust = robustify(submission, train, 'train')
    print(f"# OOF score with competition metric: {score(solution, submission_robust, ''):.4f}")

    f1_df = pd.DataFrame(f1_list, columns=['body_parts_tracked_str', 'action', 'binary F1 score'])
    print(f"# Average of {len(f1_df)} binary F1 scores {f1_df['binary F1 score'].mean():.4f}")
    # with pd.option_context('display.max_rows', 500):
    #     display(f1_df)

In [18]:
if validate_or_submit != 'validate':
    if len(submission_list) > 0:
        submission = pd.concat(submission_list)
    else:
        submission = pd.DataFrame(
            dict(
                video_id=438887472,
                agent_id='mouse1',
                target_id='self',
                action='rear',
                start_frame=278,
                stop_frame=500
            ), index=[44])
    if validate_or_submit == 'submit':
        submission_robust = robustify(submission, test, 'test')
    else:
        submission_robust = robustify(submission, stresstest, 'stresstest', 'stresstest_tracking')
    submission_robust.index.name = 'row_id'
    submission_robust.to_csv('submission.csv')
    !head submission.csv

row_id,video_id,agent_id,target_id,action,start_frame,stop_frame
0,438887472,mouse1,mouse4,submit,1427,1430
1,438887472,mouse1,mouse4,submit,1431,1434
2,438887472,mouse1,mouse4,attack,1436,1444
3,438887472,mouse1,mouse4,attack,1449,1488
4,438887472,mouse1,mouse4,submit,1491,1496
5,438887472,mouse1,mouse4,attack,1508,1514
6,438887472,mouse1,mouse4,attack,1515,1536
7,438887472,mouse1,mouse4,attack,2415,2421
8,438887472,mouse1,mouse4,avoid,2564,2591
